This Notebook will do the following:
- Will extract a csv file of respctive holders
    - This exact code will use M1 Finance built-in csv export
- After export it will add A Beta, and Standard deviation to the respective time frequency
- Note: Assumes MacOS, and file downloaded in Desktop


In [5]:
from datetime import datetime, timedelta, date
import numpy as np
import pandas as pd
import scipy.stats
import yfinance as yf

Global Varibales

In [13]:
time_frame = '5Y Monthly' 

# Change start date accordingly 
todays_date = date.today()
start_date = datetime.now() - timedelta(days=5*365)

# Put user name here
user = 'blakeuribe'
export_path = f'data/portfolio_info_{todays_date}.csv'

# Asjust file path
formatted_date = todays_date.strftime('%b-%d-%Y')
portfolio_df = pd.read_csv(f'/Users/{user}/Desktop/Holdings-{formatted_date}.csv')

Functions

In [8]:

def get_stock_data_w_returns(symbol, freq='1mo'):
    # This will return extract stock data respective to the Ticker, and add returns

    stock_data = yf.download(symbol, start=start_date, end=todays_date, interval=freq, rounding=True)
    stock_data['Returns (%)'] = (stock_data['Adj Close'].pct_change() * 100).round(2)
    stock_data.reset_index(inplace=True) 
    stock_data.dropna(inplace=True)
    return stock_data

spy_data = get_stock_data_w_returns('SPY')
spy_returns = spy_data['Returns (%)']

def monthly_beta_calc(symbol, market_returns=spy_returns):
    # This will calcuate montlhy beta
    
    monthly_data = get_stock_data_w_returns(symbol)
    stock_returns = monthly_data['Returns (%)']
    regression = scipy.stats.linregress(x=market_returns, y=stock_returns, alternative='two-sided')
    return round(regression[0], 2)


def get_monthly_std(symbol):
    # This will calculate monthly standard deviaton
    
    monthly_returns = get_stock_data_w_returns(symbol['Returns (%)'])
    stock_std = np.std(monthly_returns)
    return round(stock_std, 2)


[*********************100%%**********************]  1 of 1 completed


Df Adjustmets & Calculations

In [14]:
cols_to_convert = ['Cost Basis', 'Unrealized Gain ($)', 'Value']

for col in cols_to_convert:
    if portfolio_df[col].dtype != 'float64': 
        portfolio_df[col] = portfolio_df[col].str.replace(',', '').astype(float)

portfolio_total_value = round(np.sum(portfolio_df['Value']), 2)

# Wieght Calc
portfolio_df['Weight (%)'] = round((portfolio_df['Value']/portfolio_total_value) * 100, 2)
# portfolio_df

# Beta Calc
portfolio_df['Beta'] = [monthly_beta_calc(stock, market_returns=spy_returns) for stock in portfolio_df['Symbol']]
portfolio_df['Avg. Returns (%)'] = [round(np.mean(get_stock_data_w_returns(stock)['Returns (%)']), 2) for stock in portfolio_df['Symbol']]

# Round
for col in portfolio_df.columns:
    if col not in ['Symbol', 'Name']:
        portfolio_df[col] = portfolio_df[col].round(2)

# Rename
portfolio_df.rename(
    columns={
        'Beta':'Beta ({period})'.format(period=time_frame),
        'Avg. Returns (%)':'Avg. Returns (%) ({period})'.format(period=time_frame)
        }, 
    inplace=True
)


portfolio_df.to_csv(export_path, index=False)
portfolio_df.info()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Symbol                         11 non-null     object 
 1   Name                           11 non-null     object 
 2   Quantity                       11 non-null     float64
 3   Avg. Price                     11 non-null     float64
 4   Cost Basis                     11 non-null     float64
 5   Unrealized Gain ($)            11 non-null     float64
 6   Unrealized Gain (%)            11 non-null     float64
 7   Value                          11 non-null     float64
 8   Weight (%)                     11 non-null     float64
 9   Beta (5Y Monthly)              11 non-null     float64
 10  Avg. Returns (%) (5Y Monthly)  11 non-null     float64
dtypes: float64(9), object(2)
memory usage: 1.1+ KB


Portfolio Beta & Expected Return 


In [11]:

portfolio_beta = round(np.sum((portfolio_df['Weight (%)'] / 100) * portfolio_df['Beta ({period})'.format(period=time_frame)]), 2)
portfolio_exp_return = round(np.sum((portfolio_df['Weight (%)'] / 100) * portfolio_df['Avg. Returns (%) ({period})'.format(period=time_frame)]), 2)

print('Portfolio Beta ({period}): {value}'.format(period=time_frame, value=portfolio_beta))
print('Portfolio E[R] Monthly (%) ({period}): {value} '.format(period=time_frame, value=portfolio_exp_return))

# portfolio_df.sort_values(
#     by=None, 
#     ascending=False
# )

# portfolio_df.to_csv()
portfolio_df

Portfolio Beta (5Y Monthly): 0.93
Portfolio E[R] Monthly (%) (5Y Monthly): 1.24 


,Symbol,Name,Quantity,Avg. Price,Cost Basis,Unrealized Gain ($),Unrealized Gain (%),Value,Weight (%),Beta (5Y Monthly),Avg. Returns (%) (5Y Monthly)
0,VOO,Vanguard S&P 500 ETF,17.94,413.51,7419.13,1557.68,21.00,8976.81,32.28,1.00,1.34
1,HDV,iShares Core High Dividend ETF,53.55,97.62,5227.94,876.55,16.77,6104.49,21.95,0.82,0.81
2,VDC,Vanguard Consumer Staples ETF,19.13,187.85,3593.00,390.88,10.88,3983.88,14.33,0.61,0.80
3,XLK,Technology Select Sector SPDR Fund,12.38,188.01,2327.28,349.99,15.04,2677.27,9.63,1.13,2.00
4,XLE,Energy Select Sector SPDR Fund,21.70,82.26,1785.22,215.47,12.07,2000.69,7.19,1.24,1.83
5,SOXX,iShares PHLX Semiconductor ETF,4.95,174.31,862.24,272.33,31.58,1134.57,4.08,1.32,2.53
6,TMF,Direxion Daily 20+ Year Treasury Bull 3X Shares,14.89,51.31,764.01,-16.97,-2.22,747.04,2.69,0.83,-2.17
7,MSFT,Microsoft,1.60,382.16,610.36,68.85,11.28,679.21,2.44,0.87,2.21
8,COST,Costco Wholesale Corp.,0.77,529.48,407.50,221.75,54.42,629.25,2.26,0.77,2.10
9,XOM,Exxon Mobil Corp.,4.28,90.71,388.30,113.97,29.35,502.27,1.81,0.85,1.81
